In [ ]:
import requests
import json

with open('auth.json', 'r') as auth:
    auth_data = json.load(auth)
key = auth_data['key']
    
def get_friend_ids(id):
    r = requests.get('http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={0}&steamid={1}&relationship=friend'.format(key, id)).text
    r = json.loads(r)
    friends = r["friendslist"]["friends"]
    
    friend_ids = []
    for friend in friends:
        if (friend['friend_since'] != 0):
            friend_ids.append(friend['steamid'])
    return friend_ids

def main():
    print(get_friend_ids('76561197960435530'))
    
main()